<h2>Feature Creation</h2>

In [9]:
import time
import os
import pandas as pd
import sys
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import newspaper

from IPython.display import clear_output

In [10]:
comments = pd.read_csv('./train_comments.csv',low_memory=False)
comments[['body']] = comments[['body']].astype(str)
comments.head()

,author,body,body_html,controversiality,created,created_utc,distinguished,downs,edited,gilded,...,likes,link_id,name,num_reports,parent_id,replies,score,subreddit,ups,pid
0,SirT6,The title sort of misses the point of the stud...,"&lt;div class=""md""&gt;&lt;p&gt;The title sort ...",0,1447279564,1447250764,NaN,0,False,1,...,NaN,t3_3se6lu,t1_cwwhtv7,NaN,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",1359,science,1359,3se6lu
1,DrBiochemistry,Just want to point out that until I see a deli...,"&lt;div class=""md""&gt;&lt;p&gt;Just want to po...",0,1447277409,1447248609,NaN,0,False,0,...,NaN,t3_3se6lu,t1_cwwgxle,NaN,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",3209,science,3209,3se6lu
2,Frogblood,It's an interesting idea but the in vitro and ...,"&lt;div class=""md""&gt;&lt;p&gt;It&amp;#39;s an...",0,1447276156,1447247356,NaN,0,False,0,...,NaN,t3_3se6lu,t1_cwwggxu,NaN,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",133,science,133,3se6lu
3,mijn_ikke,Just waiting until somebody smarter than me co...,"&lt;div class=""md""&gt;&lt;p&gt;Just waiting un...",0,1447275611,1447246811,NaN,0,1447248944.0,1,...,NaN,t3_3se6lu,t1_cwwga6g,NaN,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",773,science,773,3se6lu
4,awhitt8,Yes the title is sensationalized.\n\n&gt;The m...,"&lt;div class=""md""&gt;&lt;p&gt;Yes the title i...",0,1447284967,1447256167,NaN,0,1447259263.0,0,...,NaN,t3_3se6lu,t1_cwwkopn,NaN,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",16,science,16,3se6lu


In [17]:
articles = pd.read_csv('./train_articles.csv',low_memory=False)
articles[['text']] = articles[['text']].astype(str)
articles.head()

,authors,keywords,publish_date,summary,text,url,author,created_utc,domain,downs,...,num_comments,num_reports,over_18,permalink,score,selftext,subreddit,thumbnail,title,ups
0,"[Derek Keats Flickr, Hannah Osborne, Marc Cirera]","{u'toxic': 1.01308139535, u'cancer': 1.0261627...",2015-11-10 16:00:00+00:00,Algae has been genetically engineered to kill ...,Algae has been genetically engineered to kill ...,http://www.ibtimes.co.uk/algae-genetically-eng...,the_phet,1447239366,ibtimes.co.uk,0,...,1073,NaN,False,/r/science/comments/3se6lu/algae_has_been_gene...,6705,NaN,science,http://b.thumbs.redditmedia.com/y1CGKgl69hKw-s...,Algae has been genetically engineered to kill ...,6705
1,"[Derek Keats Flickr, Hannah Osborne, Marc Cirera]","{u'toxic': 1.01308139535, u'cancer': 1.0261627...",2015-11-10 16:00:00+00:00,Algae has been genetically engineered to kill ...,Algae has been genetically engineered to kill ...,http://www.ibtimes.co.uk/algae-genetically-eng...,the_phet,1447239366,ibtimes.co.uk,0,...,1073,NaN,False,/r/science/comments/3se6lu/algae_has_been_gene...,6706,NaN,science,http://b.thumbs.redditmedia.com/y1CGKgl69hKw-s...,Algae has been genetically engineered to kill ...,6706
2,"[Derek Keats Flickr, Hannah Osborne, Marc Cirera]","{u'toxic': 1.01308139535, u'cancer': 1.0261627...",2015-11-10 16:00:00+00:00,Algae has been genetically engineered to kill ...,Algae has been genetically engineered to kill ...,http://www.ibtimes.co.uk/algae-genetically-eng...,the_phet,1447239366,ibtimes.co.uk,0,...,1073,NaN,False,/r/science/comments/3se6lu/algae_has_been_gene...,6708,NaN,science,http://b.thumbs.redditmedia.com/y1CGKgl69hKw-s...,Algae has been genetically engineered to kill ...,6708
3,[],"{u'diamond': 1.04316546763, u'laser': 1.008633...",None,"If Q-carbon is harder than diamond, why would ...","This is a collection of 0.02, 0.03 and 0.04 ca...",http://phys.org/news/2015-11-phase-carbon-diam...,skoalbrother,1448903226,phys.org,0,...,2216,NaN,False,/r/science/comments/3uvg0o/researchers_find_ne...,6777,NaN,science,http://b.thumbs.redditmedia.com/hZrhEdBoJp22oE...,"Researchers find new phase of carbon, make dia...",6777
4,[],"{u'diamond': 1.04316546763, u'laser': 1.008633...",None,"If Q-carbon is harder than diamond, why would ...","This is a collection of 0.02, 0.03 and 0.04 ca...",http://phys.org/news/2015-11-phase-carbon-diam...,skoalbrother,1448903226,phys.org,0,...,2216,NaN,False,/r/science/comments/3uvg0o/researchers_find_ne...,6772,NaN,science,http://b.thumbs.redditmedia.com/hZrhEdBoJp22oE...,"Researchers find new phase of carbon, make dia...",6772


In [14]:
#Create Comment Features
########################
#Tokenized Comments

import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
stopset = set(stopwords.words('english'))
stemmer = PorterStemmer()

def get_tokens(text):
    lowers = text.lower()
    clean = regex.sub('',lowers)
    tokens=nltk.word_tokenize(clean)
    return [w for w in tokens if not w in stopset]

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#Count Stemmed Tokens
comments['tokens'] = comments.body.apply(lambda text: Counter(stem_tokens(get_tokens(text), stemmer)))

#Comment Length
comments['comment_length'] = comments.body.apply(len)

#Number of Words
comments['n_tokens'] = comments.tokens.apply(len)

#Comment Keywords
comments['keywords'] = comments.body.apply(newspaper.nlp.keywords)

print comments.columns

Index([u'author', u'body', u'body_html', u'controversiality', u'created',
       u'created_utc', u'distinguished', u'downs', u'edited', u'gilded', u'id',
       u'likes', u'link_id', u'name', u'num_reports', u'parent_id', u'replies',
       u'score', u'subreddit', u'ups', u'pid', u'tokens', u'comment_length',
       u'n_tokens', u'keywords'],
      dtype='object')


In [18]:
#Create Article Features
#########################
#Tokens
articles['tokens'] = articles['text'].apply(lambda text: Counter(stem_tokens(get_tokens(text), stemmer)))

#Article Length
articles['article_len'] = articles['text'].apply(len)

#Number of Words
articles['n_tokens'] = articles['text'].apply(len)

articles.columns

Index([u'authors', u'keywords', u'publish_date', u'summary', u'text', u'url',
       u'author', u'created_utc', u'domain', u'downs', u'gilded', u'is_self',
       u'likes', u'media', u'id', u'num_comments', u'num_reports', u'over_18',
       u'permalink', u'score', u'selftext', u'subreddit', u'thumbnail',
       u'title', u'ups', u'tokens', u'article_len', u'n_tokens'],
      dtype='object')

Lets write the final article and comment data to file.

In [20]:
articles.to_csv('./train_articles.csv',sep=',',index=False,encoding = 'utf-8')
comments.to_csv('./train_comments.csv',sep=',',index=False,encoding='utf-8')

Now we have raw content, stemmed tokens, keywords for each comment and article. Now we can start to look at the relationships between comment content and article content. We compouted the length to normalize some of the similarities. 